# [第9章: ベクトル空間法 (I)](http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch9)
[enwiki-20150112-400-r10-105752.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2)は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．  
なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる．メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパス[enwiki-20150112-400-r100-10576.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2)を用いよ．

In [ ]:
!ls Input/enwiki-20150112-400-*.txt.bz2

## 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

* トークンの先頭と末尾に出現する次の文字を削除: `.,!?;:()[]'"`
* 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [1]:
import bz2

In [ ]:
#with bz2.open("Input/enwiki-20150112-400-r10-105752.txt.bz2") as f:
with bz2.open("Input/enwiki-20150112-400-r100-10576.txt.bz2") as f_input, open("Output/Chapter9/80.txt", "w") as f_80:
    skipped_chars = ".,!?;:()[]'\""
    for i, line in enumerate(f_input):
        replaced_tokens = []
        for token in line.decode('utf-8').replace("\n", "").split():
            start, end = 0, len(token)
            while start < end and token[start] in skipped_chars:
                start += 1
            while start < end and token[end-1] in skipped_chars:
                end -= 1
            if start == end:
                continue
            replaced_tokens.append(token[start:end])
        f_80.write(" ".join(replaced_tokens) + "\n")

## 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．  
インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [ ]:
!ls Input/country_names_81.txt

In [ ]:
with open("Output/Chapter9/80.txt") as f_80, open("Output/Chapter9/81.txt", "w") as f_81:
    country_names = None
    with open("Input/country_names_81.txt") as f:
        country_names = [line.replace("\n", "") for line in f]
    for line_80 in f_80:
        line_81 = line_80
        for country_name in country_names:
            if country_name in line_81:
                line_81 = line_81.replace(country_name, "_".join(country_name.split()))
        f_81.write(line_81)

## 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

* ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
* 単語$t$を選ぶ度に，文脈幅$d$は$\{ 1, 2, 3, 4, 5 \}$の範囲でランダムに決める．

In [1]:
import random

In [2]:
random.seed(82)

In [ ]:
with open("Output/Chapter9/81.txt") as f_81, open("Output/Chapter9/82.txt", "w") as f_82:
    for line in f_81:
        words = line.split()
        for i, t in enumerate(words):
            d = random.randrange(1, 6)
            for j in range(max(i-d, 0), min(i+d, len(words))):
                if i == j:
                    continue
                args = (t, words[j])
                f_82.write("%s\t%s\n" % args)

## 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

* $f(t, c)$: 単語$t$と文脈語$c$の共起回数
* $f(t, \ast )$: 単語$t$の出現回数
* $f( \ast , c)$: 文脈語$c$の出現回数
* $N$: 単語と文脈語のペアの総出現回数

In [3]:
import subprocess

In [ ]:
!sort Output/Chapter9/82.txt | uniq -c > Output/Chapter9/83_f_tc.txt

In [ ]:
!cut --fields=1 Output/Chapter9/82.txt | sort | uniq -c > Output/Chapter9/83_f_t.txt

In [ ]:
!cut --fields=2 Output/Chapter9/82.txt | sort | uniq -c > Output/Chapter9/83_f_c.txt

In [12]:
def f(t=None, c=None):
    args = ["grep"]

    if t is not None and c is not None:
        # f(t, c)
        # grep " t[[:space:]]c$" Output/Chapter9/83_t_c.txt
        args.append(" %s[[:space:]]%s$" % (t, c))
        args.append("Output/Chapter9/83_f_tc.txt")
    elif t is not None and c is None:
        # f(t, *)
        # grep " t$" Output/Chapter9/83_t.txt
        args.append(" %s$" % t)
        args.append("Output/Chapter9/83_f_t.txt")
    elif t is None and c is not None:
        # f(*, c)
        # grep " c$" Output/Chapter9/83_c.txt
        args.append(" %s$" % c)
        args.append("Output/Chapter9/83_f_c.txt")
    else:
        raise ValueError("f(*, *)")

    try:
        return int(subprocess.check_output(args).split()[0])
    except subprocess.CalledProcessError:
        return 0

In [ ]:
%%time
f(t="the", c="United_States")

In [ ]:
%%time
f(t="the")

In [ ]:
%%time
f(c="United_States")

In [5]:
%%time
args = ["wc", "-l", "Output/Chapter9/83_f_tc.txt"]
N = int(subprocess.check_output(args).split()[0])

N

CPU times: user 4.05 ms, sys: 0 ns, total: 4.05 ms
Wall time: 189 ms


18354205

## 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

* $f(t, c) \geq 10$ならば，$X_{tc} = \mathrm{PPMI} (t, c) = \max \left\{ \log \frac{N \times f(t, c)}{f(t, \ast ) \times f( \ast , c)} , 0 \right\}$
* $f(t, c) < 10$ならば，$X_{tc} = 0$

ここで，$\mathrm{PPMI} (t, c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は0になるので，非0の要素だけを書き出せばよい．

In [4]:
from math import log
import pickle
from scipy.sparse import csr_matrix
from tqdm import tqdm

In [ ]:
%%time
with open("Output/Chapter9/83_f_tc.txt") as f_83, open("Output/Chapter9/84_f_tc.txt", "w") as f_84:
    for line in f_83:
        elements = line.split()
        if int(elements[0]) >= 10:
            f_84.write(" ".join(elements) + "\n")

In [6]:
%%time
t_list, c_list = None, None

with open("Output/Chapter9/83_f_t.txt") as f_83_t, open("Output/Chapter9/83_f_c.txt") as f_83_c:
    t_list = [line.split()[1] for line in f_83_t]
    c_list = [line.split()[1] for line in f_83_c]

print("len(t_list)=%d, len(c_list)=%d" % (len(t_list), len(c_list)))

len(t_list)=382481, len(c_list)=382901
CPU times: user 211 ms, sys: 36.5 ms, total: 247 ms
Wall time: 248 ms


In [ ]:
%%time
X = None

with open("Output/Chapter9/84_f_tc.txt") as f_84_tc:
    data, row, col = [], [], []

    for line in tqdm(f_84_tc):
        elements = line.split()
        f_tc, t, c = int(elements[0]), elements[1], elements[2]
        f_t, f_c = f(t=t), f(c=c)
        ppmi = max(0.0, log(float(N*f_tc) / float(f_t*f_c)))
        if ppmi > 0.0:
            data.append(ppmi)
            row.append(t_list.index(t))
            col.append(c_list.index(c))

    X = csr_matrix((data, (row, col)), shape=(len(t_list), len(c_list)))

X

In [ ]:
with open("Output/Chapter9/84_X.pickle", "wb") as f_84_X:
    pickle.dump(X, f_84_X)

In [8]:
X = None

with open("Output/Chapter9/84_X.pickle", "rb") as f_84_X:
    X = pickle.load(f_84_X)

X

<382481x382901 sparse matrix of type '<class 'numpy.float64'>'
	with 144717 stored elements in Compressed Sparse Row format>

In [9]:
idx_united_states = t_list.index("United_States")
idx_united_states

233727

In [10]:
X[idx_united_states]

<1x382901 sparse matrix of type '<class 'numpy.float64'>'
	with 101 stored elements in Compressed Sparse Row format>

## 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [11]:
from sklearn.decomposition import TruncatedSVD

In [12]:
%%time
spca = TruncatedSVD(n_components=300, random_state=85).fit(X)
X_comp = spca.transform(X)

X_comp.shape

CPU times: user 3min 30s, sys: 44.3 s, total: 4min 14s
Wall time: 1min 30s


(382481, 300)

In [13]:
with open("Output/Chapter9/85_X_comp.pickle", "wb") as f_85:
    pickle.dump(X_comp, f_85)

In [14]:
X_comp = None

with open("Output/Chapter9/85_X_comp.pickle", "rb") as f_85:
    X_comp = pickle.load(f_85)

X_comp.shape

(382481, 300)

## 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [15]:
X_comp[idx_united_states].shape

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.

(300,)

In [16]:
X_comp[idx_united_states]

array([ 3.13457598e-01,  2.96968861e+00, -6.36476128e-01, -3.18611768e-01,
       -2.80688932e-01, -6.08157837e-02,  9.02046903e-02, -7.82913050e-01,
        8.55961697e-01, -4.59676998e-02,  1.64324883e+00, -9.31279891e-01,
        4.26686381e-01, -3.04092016e-01,  2.97654980e-01, -1.26470246e+00,
        1.48797728e+00, -1.89338813e+00,  1.96700854e+00,  9.51237625e-01,
        6.41593206e-01, -5.33678705e-01,  2.95409447e-01,  9.21636399e-01,
       -4.38686169e-01, -6.44582722e-01, -2.48358341e-01,  2.64791679e-01,
        2.46012089e-01, -3.39927371e-02,  3.08611610e-01, -6.12512884e-01,
        5.40496419e-01, -5.39949727e-01,  6.93305813e-01,  2.61330318e-01,
        1.11777459e-02, -4.35980864e-01, -1.93763364e-01, -9.30712995e-02,
       -7.38301973e-01, -6.20037386e-02,  5.51789699e-01,  1.00155707e-01,
       -9.03525935e-02, -8.08976103e-02, -5.72514020e-03, -8.16085157e-01,
       -2.36328937e-01,  1.81316040e-02, -3.96104702e-01, -4.86715512e-02,
       -2.48497517e-01, -

## 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [17]:
import numpy as np

In [18]:
def cossim(x: np.ndarray, y: np.ndarray):
    if np.linalg.norm(x) == 0 or np.linalg.norm(y) == 0:
        return -1
    else:
        return np.dot(x,y) / (np.linalg.norm(x)*np.linalg.norm(y))

In [19]:
idx_u_s = t_list.index("U.S")
idx_u_s

231734

In [20]:
cossim(X_comp[idx_united_states], X_comp[idx_u_s])

0.8293433136280866

## 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [21]:
def print_top_10(x: np.ndarray):
    cos_sims = [cossim(x,X_comp[i]) for i in tqdm(range(len(t_list)))]
    indices = np.argsort(cos_sims)
    for i in range(-1, -11, -1):
        args = (-i, t_list[indices[i]], cos_sims[indices[i]])
        print("[%d/10] %s : %f" % args)

In [22]:
idx_england = t_list.index("England")
idx_england

94156

In [23]:
print_top_10(X_comp[idx_england])

100%|██████████| 382481/382481 [00:03<00:00, 102971.49it/s]


[1/10] England : 1.000000
[2/10] Scotland : 0.693036
[3/10] Spain : 0.540434
[4/10] Wales : 0.533111
[5/10] Germany : 0.506487
[6/10] Italy : 0.505202
[7/10] Rome : 0.488755
[8/10] Hungary : 0.484105
[9/10] France : 0.475182
[10/10] Patriots : 0.469502


## 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [24]:
args = (t_list.index("Spain"), t_list.index("Madrid"), t_list.index("Athens"))
args

(214548, 151562, 48137)

In [25]:
x_89 = X_comp[args[0]] - X_comp[args[1]] + X_comp[args[2]]
print_top_10(x_89)

100%|██████████| 382481/382481 [00:03<00:00, 105343.30it/s]

[1/10] Spain : 0.843044
[2/10] Sweden : 0.815025
[3/10] Austria : 0.786050
[4/10] Belgium : 0.771359
[5/10] Netherlands : 0.767125
[6/10] Vichy : 0.765154
[7/10] Télévisions : 0.765154
[8/10] Turkey : 0.762232
[9/10] Hamburg : 0.749733
[10/10] Bavaria : 0.749257
